In [ ]:
# import PCA
from sklearn.decomposition import PCA
base_cadastral_pca = PCA(n_components=3).fit_transform(base_cadastral_scaled)
print(base_cadastral_pca.shape)

# use UMAP to reduce dimensionality
import umap
base_cadastral_umap = umap.UMAP(n_components=3).fit_transform(base_cadastral_scaled)
print(base_cadastral_umap.shape)

In [ ]:
# plot the PCA
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(base_cadastral_pca[:, 0], base_cadastral_pca[:, 1], base_cadastral_pca[:, 2], c=c_column, cmap='coolwarm')
plt.title('PCA')
plt.show()

# plot the UMAP
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(base_cadastral_umap[:, 0], base_cadastral_umap[:, 1], base_cadastral_umap[:, 2], c=c_column, cmap='coolwarm')
plt.title('UMAP')
plt.show()

In [ ]:
# show that ddd and cep_2_dig are not good features using chi2 test
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
X = base_cadastral_scaled.drop(['fraud'], axis=1)
y = base_cadastral_scaled['fraud']  
chi2_selector = SelectKBest(chi2, k=2)
X_kbest = chi2_selector.fit_transform(X, y)
print(X_kbest.shape)
print(chi2_selector.get_support(indices=True))

In [ ]:
def is_fraud_dependent_of_the_data_exclusive_to_pagamentos_table(info, pagamentos):
  info = info.copy()
  pagamentos = pagamentos.copy()

  # lable as 1 all rows of pagamentos where the pair (id_cliente, safra_ref) is in info and as 0 otherwise
  pagamentos['id_cliente_safra_ref'] = pagamentos['id_cliente'].astype(str) + '_' + pagamentos['safra_ref'].astype(str)
  info['id_cliente_safra_ref'] = info['id_cliente'].astype(str) + '_' + info['safra_ref'].astype(str)
  pagamentos['coherent'] = np.where(pagamentos['id_cliente_safra_ref'].isin(info['id_cliente_safra_ref']), 1, 0)

  # Chi-square test to determine if the fraud is independent of the coherent column
  from scipy.stats import chi2_contingency
  contingency_table = pd.crosstab(pagamentos['fraud'], pagamentos['coherent'])
  stat, p, dof, expected = chi2_contingency(contingency_table)

  if p < 0.05:
    print('The fraud is dependent of the data in pagamentos where its key does not appear in the info table.\n' +
          'This data should be kept in the pagamentos table')
  else:
    print('The fraud is independent of the data in pagamentos where its key does not appear in the info table\n' +
          'This data can be removed from the pagamentos table')
  return p < 0.05

if not is_fraud_dependent_of_the_data_exclusive_to_pagamentos_table(base_info_date, base_pagamentos_date):
  def clean_pagamentos_table(info, pagamentos):
    # Create a DataFrame that represents the primary key of base_info
    base_info_keys = base_info_date[['id_cliente', 'safra_ref']]

    # Merge with base_pagamentos, keeping only the records with matching keys
    base_pagamentos_coherent = pd.merge(base_pagamentos_date, base_info_keys, on=['id_cliente', 'safra_ref'], how='inner')

    # Count the number of excluded rows
    excluded_rows = len(base_pagamentos_date) - len(base_pagamentos_coherent)
    print(f'Number of excluded rows: {excluded_rows}')
    print(f'Percentage of excluded rows: {excluded_rows / len(base_pagamentos_date) * 100:.2f}%')

    # Save the coherented DataFrame to a new CSV file
    # base_pagamentos_coherent.to_csv('base_pagamentos_desenvolvimento_coherent.csv', index=False)

    print('The pagamentos table was cleaned successfully')
    return base_pagamentos_coherent
  base_pagamentos_date = clean_pagamentos_table(base_info_date, base_pagamentos_date)

In [ ]:
# plot a gaussian distribution of late payments for payments within -20 and 20 days of delay
plt.figure(figsize=(10, 6))
sns.histplot(base_pagamentos_date[(base_pagamentos_date['late_payment'] >= -20) & (base_pagamentos_date['late_payment'] <= 20)]['late_payment'], bins=np.arange(-20.5, 20.5, 1), kde=True, log_scale=(False, True))
plt.title('Late payments')
plt.xlabel('Days')
plt.ylabel('Frequency')
plt.show()


In [ ]:
base_info_date['safra_ref'].value_counts().sort_index().plot(kind='bar', figsize=(10, 6))

In [ ]:
# from create_sheets import create_spreadsheet, upload_csv_to_sheet

# spreadsheet_id = create_spreadsheet('Test Spreadsheet')
# upload_csv_to_sheet(spreadsheet_id, 'base_pagamentos_sorted_id_emissao_pagamento.csv', 'Sheet1')
# upload_csv_to_sheet(spreadsheet_id, 'base_pagamentos_drop_dupl.csv', 'Sheet2')
# # print the spreadsheet link
# print(f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/edit")

In [ ]:
# # grafico de linha do 50% percentile de transacoes por cliente, transacoes por cliente fraudulento e transacoes fraudulentas por cliente fraudulento
# plt.figure(figsize=(10, 6))

# def concatenate_descriptions(dataframes, names):
#     descriptions = []
    
#     for df, name in zip(dataframes, names):
#         description = df.describe().to_frame().T
#         description.index = [name]
#         descriptions.append(description)
    
#     concatenated_descriptions = pd.concat(descriptions).T
#     return concatenated_descriptions

# dataframes = [transacoes_por_cliente, transacoes_por_cliente_fraudulento, fraudulent_transacoes_por_cliente_fraudulento]
# names = ['transacoes_por_cliente', 'transacoes_por_cliente_fraudulento', 'fraudulent_transacoes_por_cliente_fraudulento']
# transacoes_descriptions = concatenate_descriptions(dataframes, names)

# # plot the line graph
# sns.lineplot(data=transacoes_descriptions.loc[['25%', '50%', '75%'], :], dashes=False)
# plt.title('Number of transactions per client')
# plt.xlabel('Percentile')
# plt.ylabel('Number of transactions')
# plt.show()

# transacoes_descriptions.head(10)
# # base_pagamentos_date.groupby("id_cliente")["fraud"].sum()
# # base_pagamentos_date[base_pagamentos_date["id_cliente"] == 209314261782935157]['fraud']

In [ ]:
# clientes aparecem em base_info_date mas não em base_pagamentos_date
clientes_info = base_info_date['id_cliente'].unique()
clientes_pagamentos = base_pagamentos_date['id_cliente'].unique()
clientes_info_not_in_pagamentos = [cliente for cliente in clientes_info if cliente not in clientes_pagamentos]
clientes_pagamentos_not_in_info = [cliente for cliente in clientes_pagamentos if cliente not in clientes_info]
print(f'Number of clientes in base_info but not in base_pagamentos: {len(clientes_info_not_in_pagamentos)}')
print(f'Percentage of clientes in base_info but not in base_pagamentos: {len(clientes_info_not_in_pagamentos) / len(clientes_info) * 100:.2f}%\n')
print(f'Number of clientes in base_pagamentos but not in base_info: {len(clientes_pagamentos_not_in_info)}')
print(f'Percentage of clientes in base_pagamentos but not in base_info: {len(clientes_pagamentos_not_in_info) / len(clientes_pagamentos) * 100:.2f}%\n')

# porcentage of transactions marked as fraudulent
print(f'Percentage of transactions marked as fraudulent: {base_pagamentos_date["fraud"].sum() / len(base_pagamentos_date) * 100:.2f}%')

# porcentage of clients marked as fraudulent
print(f'Percentage of clients marked as fraudulent at least once: {base_pagamentos_date.groupby("id_cliente")["fraud"].any().sum() / len(base_pagamentos_date["id_cliente"].unique()) * 100:.2f}%\n')

# quantidade de clientes
print(f'Number of clients: {len(base_pagamentos_date["id_cliente"].unique())}\n')

# quantas transações cada cliente tem em base_pagamentos_date
transacoes_por_cliente = base_pagamentos_date.groupby('id_cliente').size().sort_values(ascending=False)
print(f'Statistics of the number of transactions per client:\n{transacoes_por_cliente.describe()}\n')

# quantas transações cada cliente fraudulento tem em base_pagamentos_date
transacoes_por_cliente_fraudulento = base_pagamentos_date.groupby('id_cliente').agg({'fraud': 'any', 'id_cliente': 'count'})
transacoes_por_cliente_fraudulento.columns = ['fraud', 'transacoes']
transacoes_por_cliente_fraudulento = transacoes_por_cliente_fraudulento[transacoes_por_cliente_fraudulento['fraud'] == 1]['transacoes'].sort_values(ascending=False)
print(f'Statistics of the number of transactions per fraudulent client:\n{transacoes_por_cliente_fraudulento.describe()}\n')

# quantas transações fraudulentas cada cliente fraudulento tem em base_pagamentos_date
fraudulent_transacoes_por_cliente_fraudulento = base_pagamentos_date[base_pagamentos_date['fraud'] == 1].groupby('id_cliente').size().sort_values(ascending=False)
print(f'Statistics of the number of fraudulent transactions per fraudulent client:\n{fraudulent_transacoes_por_cliente_fraudulento.describe()}\n')

# porcentage of the first transaction of each client that is fraudulent
print(f"Percentage of the first transaction that is fraudulent: {base_pagamentos_date.sort_values(['id_cliente', 'data_emissao_documento']).drop_duplicates('id_cliente').fraud.mean() * 100:.2f}%\n")